In [3]:
# Next: Final player choice locked in
# player choice v computer choice comparison
# winner declaration

import cv2
from keras.models import load_model
import numpy as np
import random
import time

def detectClick(event, x, y, flags, param):
    global gameIntro
    global countdown
    if event == cv2.EVENT_LBUTTONDOWN and gamePlay == False:
        gameIntro = True
        countdown = currentTime + 5

def createPlayerChoice():
    global solidPlayerChoice
    # choose the best probable value from a buffer of 12 previous lines
    maxValue = np.argmax(prediction)
    playerChosen = (playerChoice[maxValue])

    if len(bufferPlayerChoice) < 12:
        bufferPlayerChoice.append(playerChosen)
    else:
        bufferPlayerChoice.pop(0)
        bufferPlayerChoice.append(playerChosen)
    
    # "Rock" choice is a bit fragile, so accept "Rock" as the best option if 2 or more in buffer
    if bufferPlayerChoice.count("Rock") >= 2:
        solidPlayerChoice = "Rock"
    else:
        solidPlayerChoice = max(bufferPlayerChoice, key = bufferPlayerChoice.count)

def winnerEval(comp, player):
    global cWins
    global pWins
    global round
    global updated
    if comp == "Rock" and player == "Scissors":
        winner = "C"
    elif comp == "Paper" and player == "Rock":
        winner = "C"
    elif comp == "Scissors" and player == "Paper":
        winner = "C"
    elif player == "-Waiting-":
        winner = "Error"
    else:
        winner = "P"
    if updated == False:
        if winner == "C":
            cWins += 1
            round += 1
        elif winner == "P":
            pWins += 1
            round += 1
        updated = True
    print(round)
    return winner

def placeholderGenerate():
    global round
    createPlayerChoice()
    # Make the screen pretty with boxes and labels added to the frame
    cv2.rectangle(frame, (10,10), (442, 50), (255,255,255), -1)
    message = "Round: " + str(round) 
    cv2.putText(frame, message, (55,380), 1, 2, (0,0,0))
    # White box background
    cv2.rectangle(frame, (10,430), (440, 350), (255,255,255), -1)
    # Player label
    cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
    # Computer label
    cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
    # vs separator
    cv2.putText(frame, "V", (190,420), 2, 3, (0,0,0))
    # Create the text of the interpretted choice from the buffer
    cv2.putText(frame, solidPlayerChoice, (20,420), 0, 1, (0,0,0))
    # Green Button
    cv2.rectangle(frame, (175,80), (275, 120), (20,170,20), -1)
    # Begin text
    cv2.putText(frame, "Click", (180,110), 3, 1, (0,0,0))

def randCompChoice():
    global compChoice
    global choiceFlag
    compChoice = random.choice(playerChoice[0:3])

def gameIntroGenerate():
    createPlayerChoice()
    global choiceFlag
    global gameIntro
    global gamePlay
    if choiceFlag == False:
        randCompChoice()
        choiceFlag = True
    if gameIntro:
        cv2.rectangle(frame, (10,430), (440, 350), (0,255,255), -1)
        # Player label
        cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
        # Computer label
        cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
        # vs separator
        cv2.putText(frame, str(int(countdown-currentTime)), (190,420), 2, 3, (0,0,0))
        # Create the text of the interpretted choice from the buffer
        cv2.putText(frame, solidPlayerChoice, (20,420), 0, 1, (0,0,0))
        # Create the text of the computer's random choice
        cv2.putText(frame, compChoice, (280,420), 0, 1, (0,0,0))
        if int(countdown-currentTime) <= 0:
            gameIntro = False
            gamePlay = True
            gamePlayGenerate()   
    

def gamePlayGenerate():
    global first
    gameIntro == False
    gamePlay == True
    print("PLAYING!!!!")
        # White box background
    cv2.rectangle(frame, (10,430), (440, 350), (200,200,200), -1)
    # Player label
    cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
    # Computer label
    cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
    # vs separator
    cv2.putText(frame, "V", (190,420), 2, 3, (0,0,0))
    # Create the text of the interpretted choice from the buffer
    cv2.putText(frame, solidPlayerChoice, (20,420), 4, 1, (0,0,0))
    # Create the text of the computer's random choice
    cv2.putText(frame, compChoice, (280,420), 0, 1, (0,0,0))
    # Green Button
    cv2.rectangle(frame, (100,80), (352, 120), (20,170,20), -1)
    
    # Actually decide on the winner
    if winnerEval(compChoice, solidPlayerChoice) == "C":
        cv2.putText(frame, "CPU Wins!", (150,110), 3, 1, (0,0,0))
    elif winnerEval(compChoice, solidPlayerChoice) == "P":
        cv2.putText(frame, "Player Wins!", (130,110), 3, 1, (0,0,0))
    else:
        cv2.putText(frame, "Too Slow", (150,110), 3, 1, (0,0,0))
    

# Declare variables

model = load_model('keras_model4.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
playerChoice = ["Rock", "Paper", "Scissors", "-Waiting-", "Play Game"]

bufferPlayerChoice = []
solidPlayerChoice = ""
gameIntro = False
gamePlay = False
countdown = 3
compChoice = "EMPTY"
choiceFlag = False
first = True
pWins = 0
cWins = 0
round = 1
updated = False


while True: 
    # I don't know what ret is. It doesn't appear anywhere else
    ret, frame = cap.read()
    # crop the frame to match teachablemachine's cropped area
    if frame is None:
        print("No Image")
        continue # move to the top of the while loop without any further processing of this iteration
    else:
        frame = frame[32:, 188:]

    # resize the frame to 224 x 224
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)
    
    # Output the best choice to the shell
    print(solidPlayerChoice)
    print(compChoice)
    print(time.time())
    currentTime = time.time()

    # Generates appropriate Display overlay
    if gameIntro:
        gameIntroGenerate()
    elif gamePlay == True and gameIntro == False:
        gamePlayGenerate()
    else:
        placeholderGenerate()

    # Finally, output the frame window, with a nice title
    cv2.namedWindow("Rock Paper Scissors")
    cv2.imshow("Rock Paper Scissors", frame)

    # Click mouse anywhere to begin
    cv2.setMouseCallback("Rock Paper Scissors", detectClick)

    # Press q to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    


            
# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()


EMPTY
1643453338.1171453
-Waiting-
EMPTY
1643453338.1487026
-Waiting-
EMPTY
1643453338.1795423
-Waiting-
EMPTY
1643453338.222634
-Waiting-
EMPTY
1643453338.250122
-Waiting-
EMPTY
1643453338.2766662
-Waiting-
EMPTY
1643453338.3047664
-Waiting-
EMPTY
1643453338.3323722
-Waiting-
EMPTY
1643453338.35968
-Waiting-
EMPTY
1643453338.3878634
-Waiting-
EMPTY
1643453338.419023
-Waiting-
EMPTY
1643453338.4463253
-Waiting-
EMPTY
1643453338.472761
-Waiting-
EMPTY
1643453338.4999123
-Waiting-
EMPTY
1643453338.5284255
-Waiting-
EMPTY
1643453338.556749
-Waiting-
EMPTY
1643453338.5838647
-Waiting-
EMPTY
1643453338.611732
-Waiting-
EMPTY
1643453338.6437569
-Waiting-
EMPTY
1643453338.6711905
-Waiting-
EMPTY
1643453338.6983078
-Waiting-
EMPTY
1643453338.7261376
-Waiting-
EMPTY
1643453338.7559285
-Waiting-
EMPTY
1643453338.7880871
-Waiting-
EMPTY
1643453338.8201413
-Waiting-
EMPTY
1643453338.8518376
-Waiting-
EMPTY
1643453338.8925633
-Waiting-
EMPTY
1643453338.920176
-Waiting-
EMPTY
1643453338.9564505
-Wa